In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import scipy.io
import numpy as np
from pandas import DataFrame
import collocation

In [2]:
from matplotlib import rcParams
rcParams.update({
    "font.size": 14})

## Load in FEM Test Data

In [3]:
burgers = scipy.io.loadmat('matlab/burgers.mat')
X_ = torch.from_numpy(burgers['X']).reshape(-1).float()
T_ = torch.from_numpy(burgers['T']).reshape(-1).float()
u_ = torch.from_numpy(burgers['u']).reshape(-1).float()
X_.requires_grad = True
T_.requires_grad = True
xmin = X_.min().item()
xmax = X_.max().item()
N_x = X_.shape[0]

In [4]:
u_test = u_.reshape(X_.shape[0], T_.shape[0])
x_test, t_test = torch.meshgrid(X_, T_, indexing='ij')
x_test = x_test.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

## Problem Definition

### Initial and Boundary Conditions

In [5]:
_u_i = lambda x: -torch.sin(torch.pi*x)
_u_b = lambda t: 0*t

# Network Setup

In [6]:
from BurgersNet import Net

In [7]:
def MSE(net, u_, x, t):
    u = u_.detach().reshape(-1,1)
    u_hat = net(x,t).detach().reshape(-1,1)
    MSE = torch.mean((u - u_hat)**2)
    return MSE

In [8]:
def progress(i, total):
    print("\r", '{:.2f}% '.format((i+1)*100/total), end="")

## Hyperparameter Search

In [9]:
from model_caching import get_model_name

Baseline model

In [10]:
N_hid = 30
N_layers = 3
learning_rate = 0.001
act_func = nn.Tanh()
epochs = 5000

In [11]:
def train(net, epochs, colloc_method = 'random', n = 30, n_i = 50, n_b = 50, n_val=30, n_i_val=50, n_b_val=50):
    loss_train, loss_val, loss_test = torch.zeros(epochs), torch.zeros(epochs), torch.zeros(epochs)
    x_val, t_val, x_i_val, t_i_val, u_i_val, x_b_val, t_b_val, u_b_val = collocation.random(n_val, n_i_val, n_b_val, _u_i, _u_b, xmin, xmax)

    if colloc_method == 'uniform':
        x, t, x_i, t_i, u_i, x_b, t_b, u_b = collocation.uniform(n, n_i, n_b, _u_i, _u_b, xmin, xmax)
    if colloc_method == 'hypercube':
        x, t, x_i, t_i, u_i, x_b, t_b, u_b = collocation.hypercube(n, n_i, n_b, _u_i, _u_b, xmin, xmax)
    for e in range(epochs):
        if colloc_method == 'random':
            x, t, x_i, t_i, u_i, x_b, t_b, u_b = collocation.random(n, n_i, n_b, _u_i, _u_b, xmin, xmax)
        loss_pde, loss_bc, loss_ic = net.step(x, t, x_b, t_b, u_b, x_i, t_i, u_i)
        loss_train[e] = loss_pde + loss_bc + loss_ic
        loss_validation, *_ = net.loss(x_val, t_val, x_b_val, t_b_val, u_b_val, x_i_val, t_i_val, u_i_val)
        loss_val[e] = loss_validation.detach()
        loss_test[e] = MSE(net, u_test, x_test, t_test)
        progress(e, epochs)
        if (e+1)%500 == 0:
            print('\nEpoch: {:d}. Train loss: {:.3f}. Val Loss: {:.3f}'.format(e+1, loss_train[e], loss_val[e]))
    return loss_train, loss_val, loss_test

In [12]:
lrs = [0.0005, 0.001, 0.005, 0.01]

for lr in lrs:
    print(lr)
    net = Net(N_in=2, N_out=1, N_hid=N_hid, N_layers=N_layers,
        learning_rate=lr, act_func=act_func)
    losses = train(net, epochs)
    torch.save(losses, './model/' + get_model_name(N_hid, N_layers, lr, act_func, epochs) + '_losses.pt')

0.0005
 10.00% .94% 
Epoch: 500. Train loss: 0.159. Val Loss: 0.168
 20.00% 
Epoch: 1000. Train loss: 0.150. Val Loss: 0.156
 30.00% 
Epoch: 1500. Train loss: 0.141. Val Loss: 0.145
 40.00% 
Epoch: 2000. Train loss: 0.111. Val Loss: 0.131
 50.00% 
Epoch: 2500. Train loss: 0.113. Val Loss: 0.124
 60.00% 54.08% 
Epoch: 3000. Train loss: 0.099. Val Loss: 0.120
 70.00% 
Epoch: 3500. Train loss: 0.117. Val Loss: 0.116
 80.00% 
Epoch: 4000. Train loss: 0.104. Val Loss: 0.113
 90.00% 
Epoch: 4500. Train loss: 0.094. Val Loss: 0.110
 100.00% 
Epoch: 5000. Train loss: 0.106. Val Loss: 0.107
0.001
 10.00% 
Epoch: 500. Train loss: 0.140. Val Loss: 0.134
 20.00% 18.52% 
Epoch: 1000. Train loss: 0.127. Val Loss: 0.111
 30.00% 20.76% 22.24% 
Epoch: 1500. Train loss: 0.128. Val Loss: 0.100
 40.00% 
Epoch: 2000. Train loss: 0.111. Val Loss: 0.091
 50.00% 
Epoch: 2500. Train loss: 0.098. Val Loss: 0.082
 60.00% 
Epoch: 3000. Train loss: 0.085. Val Loss: 0.077
 70.00% 
Epoch: 3500. Train loss: 0.078. Va

In [13]:
act_funcs = [nn.Tanh(), nn.Sigmoid(), nn.Tanhshrink(), nn.ReLU()]

for act in act_funcs:
    print(str(act)[:-2])
    net = Net(N_in=2, N_out=1, N_hid=N_hid, N_layers=N_layers,
        learning_rate=learning_rate, act_func=act)

    losses = train(net, epochs)
    torch.save(losses, './model/' + get_model_name(N_hid, N_layers, learning_rate, act, epochs) + '_losses.pt')

Tanh
 10.00% .98% 4.32% 
Epoch: 500. Train loss: 0.151. Val Loss: 0.122
 20.00% 
Epoch: 1000. Train loss: 0.114. Val Loss: 0.109
 30.00% 
Epoch: 1500. Train loss: 0.098. Val Loss: 0.096
 40.00% 
Epoch: 2000. Train loss: 0.095. Val Loss: 0.089
 50.00% 
Epoch: 2500. Train loss: 0.095. Val Loss: 0.083
 60.00% 
Epoch: 3000. Train loss: 0.076. Val Loss: 0.078
 70.00% 
Epoch: 3500. Train loss: 0.078. Val Loss: 0.070
 80.00% 70.66% 
Epoch: 4000. Train loss: 0.078. Val Loss: 0.061
 90.00% 
Epoch: 4500. Train loss: 0.050. Val Loss: 0.050
 100.00% 
Epoch: 5000. Train loss: 0.045. Val Loss: 0.043
Sigmoid
 10.00% .38% 9.66% 
Epoch: 500. Train loss: 0.333. Val Loss: 0.411
 20.00% 13.12% 13.64% 
Epoch: 1000. Train loss: 0.351. Val Loss: 0.381
 30.00% 20.08% 24.84% 
Epoch: 1500. Train loss: 0.247. Val Loss: 0.248
 40.00% 32.58% 36.08% 
Epoch: 2000. Train loss: 0.143. Val Loss: 0.147
 50.00% 41.32% 
Epoch: 2500. Train loss: 0.130. Val Loss: 0.127
 60.00% 
Epoch: 3000. Train loss: 0.134. Val Loss: 0.11

In [14]:
layers = [1, 2, 3, 4]

for N_l in layers:
    print(N_l)
    net = Net(N_in=2, N_out=1, N_hid=N_hid, N_layers=N_l,
        learning_rate=learning_rate, act_func=act_func)
    losses = train(net, epochs)
    torch.save(losses, './model/' + get_model_name(N_hid, N_l, learning_rate, act_func, epochs) + '_losses.pt')

1
 10.00% .32% 
Epoch: 500. Train loss: 0.377. Val Loss: 0.360
 20.00% 11.26% 11.54% 
Epoch: 1000. Train loss: 0.212. Val Loss: 0.198
 30.00% 
Epoch: 1500. Train loss: 0.167. Val Loss: 0.150
 40.00% 32.62% 34.02% 
Epoch: 2000. Train loss: 0.145. Val Loss: 0.150
 50.00% 
Epoch: 2500. Train loss: 0.149. Val Loss: 0.150
 60.00% 
Epoch: 3000. Train loss: 0.150. Val Loss: 0.151
 70.00% 
Epoch: 3500. Train loss: 0.150. Val Loss: 0.149
 80.00% 
Epoch: 4000. Train loss: 0.161. Val Loss: 0.149
 90.00% 
Epoch: 4500. Train loss: 0.141. Val Loss: 0.150
 100.00% 
Epoch: 5000. Train loss: 0.168. Val Loss: 0.148
2
 10.00% .70% 
Epoch: 500. Train loss: 0.174. Val Loss: 0.185
 20.00% 14.92% 15.44% 
Epoch: 1000. Train loss: 0.150. Val Loss: 0.138
 30.00% 
Epoch: 1500. Train loss: 0.124. Val Loss: 0.130
 40.00% 
Epoch: 2000. Train loss: 0.124. Val Loss: 0.121
 50.00% 
Epoch: 2500. Train loss: 0.115. Val Loss: 0.122
 60.00% 55.66% 
Epoch: 3000. Train loss: 0.117. Val Loss: 0.117
 70.00% 66.30% 
Epoch: 350

In [15]:
N_hids = [10, 30, 50, 100]

for N_h in N_hids:
    print(N_h)
    net = Net(N_in=2, N_out=1, N_hid=N_h, N_layers=N_layers,
        learning_rate=learning_rate, act_func=act_func)

    losses = train(net, epochs)
    torch.save(losses, './model/' + get_model_name(N_h, N_layers, learning_rate, act_func, epochs) + '_losses.pt')

10
 10.00% 
Epoch: 500. Train loss: 0.215. Val Loss: 0.209
 20.00% 
Epoch: 1000. Train loss: 0.146. Val Loss: 0.157
 30.00% 23.70% 
Epoch: 1500. Train loss: 0.129. Val Loss: 0.144
 40.00% 38.34% 
Epoch: 2000. Train loss: 0.124. Val Loss: 0.135
 50.00% 
Epoch: 2500. Train loss: 0.108. Val Loss: 0.128
 60.00% 
Epoch: 3000. Train loss: 0.118. Val Loss: 0.123
 70.00% 
Epoch: 3500. Train loss: 0.115. Val Loss: 0.117
 80.00% 73.64% 
Epoch: 4000. Train loss: 0.108. Val Loss: 0.112
 90.00% 
Epoch: 4500. Train loss: 0.105. Val Loss: 0.108
 100.00% 
Epoch: 5000. Train loss: 0.083. Val Loss: 0.104
30
 10.00% .88% 3.18% 3.44% 8.34% 9.96% 
Epoch: 500. Train loss: 0.162. Val Loss: 0.157
 20.00% 10.76% 
Epoch: 1000. Train loss: 0.127. Val Loss: 0.128
 30.00% 20.98% 
Epoch: 1500. Train loss: 0.124. Val Loss: 0.102
 40.00% 
Epoch: 2000. Train loss: 0.112. Val Loss: 0.093
 50.00% 
Epoch: 2500. Train loss: 0.096. Val Loss: 0.078
 60.00% 
Epoch: 3000. Train loss: 0.100. Val Loss: 0.078
 70.00% 
Epoch: 350

# Train and Inspect Optimal network

In [16]:
# N_hid = 30
# N_layers = 3
# learning_rate = 0.001
# act_func = nn.Tanh()
# epochs = 5000

# net = Net(N_in=2, N_out=1, N_hid=N_hid, N_layers=N_layers,
#     learning_rate=learning_rate, act_func=act_func)

# train(net, epochs)

**(Optional) Cash model and results**

Save the NN model to disk (for caching)

In [17]:
# from model_caching import save_model, load_model, get_model_name

# model_name = get_model_name(N_hid, N_layers, learning_rate, act_func, epochs)
# save_model(net, loss_data, loss_pde, loss_bc, loss_ic, epochs)

Load the model in from disk (for evaluation)

In [18]:
# net, losses = load_model(N_hid, N_layers, learning_rate, act_func, epochs)

### Inspect Model

In [19]:
# fig, ax = plt.subplots(figsize=(5,4))
# ax.semilogy(loss_pde.detach(), label='PDE loss')
# ax.semilogy(loss_bc.detach(), label='BC loss')
# ax.semilogy(loss_ic.detach(), label='IC loss')
# ax.semilogy(loss_data, label='Evaluation loss')
# ax.legend()
# ax.set_ylabel('MSE')
# ax.set_xlabel('Epoch')
# ax.grid()
# fig.tight_layout()
# # fig.savefig('./figs/hyperparams/Burger_loss_' + model_name + '.png', dpi=600)

In [20]:
# ## Evaluate network
# fig, axs = plt.subplots(2,2,figsize=(8,6))
# axs = axs.flatten()
# u_hat = net(x_eval, t_eval).detach().reshape(u_.shape)
# pde_hat = net.PDE(x_eval, t_eval).detach().reshape(u_.shape)
# vmin = -1 # u_hat.min()
# vmax = 1 # u_hat.max()

# levels = torch.linspace(vmin, vmax, 100)
# c_levels = torch.linspace(vmin, vmax, 5)

# m = axs[0].contourf(t_.detach(), x_.detach(), u_.detach(), levels=levels, cmap='RdYlBu')
# cb = fig.colorbar(m, ax=axs[0])
# cb.set_ticks(c_levels)
# axs[0].set_title('FEM solution')
# # axs[0].set_xlabel('$t$')
# axs[0].set_ylabel('$x$')

# m = axs[1].contourf(t_.detach(), x_.detach(), u_hat.detach(), levels=levels, cmap='RdYlBu')
# cb = fig.colorbar(m, ax=axs[1])
# cb.set_ticks(c_levels)
# axs[1].set_title('PINN solution')
# # axs[1].set_xlabel('$t$')
# # axs[1].set_ylabel('$x$')

# vmin = -np.round((u_-u_hat).abs().max(),1)
# vmax = -vmin
# levels = torch.linspace(vmin, vmax, 100)
# c_levels = torch.linspace(vmin, vmax, 5)

# m = axs[2].contourf(t_.detach(), x_.detach(), (u_-u_hat).detach(), levels=levels, cmap='RdGy')
# cb = fig.colorbar(m, ax=axs[2])
# cb.set_ticks(c_levels)
# axs[2].set_title('Residual')
# axs[2].set_xlabel('$t$')
# axs[2].set_ylabel('$x$')

# vmin = -np.round(pde_hat.abs().max(),0)
# vmax = -vmin
# levels = torch.linspace(vmin, vmax, 100)
# c_levels = torch.linspace(vmin, vmax, 5)
# m = axs[3].contourf(t_.detach(), x_.detach(), pde_hat.detach(), levels=levels, vmin=vmin, vmax=vmax, cmap='RdGy')
# cb = fig.colorbar(m, ax=axs[3])
# cb.set_ticks(c_levels)
# axs[3].set_title('PDE residual')
# axs[3].set_xlabel('$t$')
# # axs[3].set_ylabel('$x$')

# fig.tight_layout()
# # fig.savefig('./figs/hyperparams/burgers_2D_' + model_name + '.png', dpi=600)


In [21]:
# xi, ti = torch.meshgrid(torch.linspace(xmin,xmax,N_x, requires_grad=True), torch.tensor([0.25, 0.5, 0.75, 1]))
# fig, axs = plt.subplots(1,4,figsize=(8,2.2))
# for k, ax in enumerate(axs):
#     x = xi[:, k].reshape(-1,1)
#     t = ti[:, k].reshape(-1,1)
#     u_hat = net(x, t).detach()
#     ax.plot(x.detach(), u_[:, (k+1)*25], color='blue', ls='-')
#     ax.plot(x.detach(), u_hat, color='red', ls='--')
#     ax.set_xlabel('$x$')
#     ax.set_title('t = {:.2f}'.format(t[0].item()))
#     ax.set_ylim([-1-0.1,1+0.1])
#     ax.grid()
# axs[0].set_ylabel('$u(x,t)$')
# fig.legend(['FEM', 'PINN'], loc="lower center", bbox_to_anchor=(0.53, -0.2))
# fig.tight_layout()
# # fig.savefig('./figs/hyperparams/burger_eval_1D_' + model_name + '.pdf', bbox_inches='tight')